In [ ]:
import json
import random
import os
import numpy as np
from pathlib import Path
from typing import List, Dict, Any, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================

# Diretório onde estão os JSONs de TREINO (Excluindo o holdout set)
TREINO_DIR = Path('data/json/llm_ready')

# Arquivo para salvar o índice de embeddings (Numpy array e a lista de fatores)
INDICE_PATH = Path('data/fewshot_index.json')

# ==============================================================================
# FUNÇÕES DE PROCESSAMENTO
# ==============================================================================

def criar_indice_tfidf(treino_dir: Path) -> Tuple[Any, List[Dict[str, Any]]]:
    """
    Cria um índice baseado em TF-IDF (simulação do Embedding) e coleta dados.
    """
    documentos = []
    metadata = []
    arquivos_json = list(treino_dir.glob("*.json"))

    print(f"📁 Preparando para indexar {len(arquivos_json)} documentos de treino...")

    for arquivo_path in arquivos_json:
        try:
            with open(arquivo_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Concatena Historico e Analise para criar o documento (contexto)
            historico = data.get('conteudo', {}).get('historico_voo', '')
            analise = data.get('conteudo', {}).get('analise', '')
            texto_contexto = historico + " " + analise

            fatores = data.get('conteudo', {}).get('fatores_contribuintes', [])

            if len(fatores) > 0 and len(texto_contexto) > 50:
                documentos.append(texto_contexto)
                # Armazena o JSON original, o texto de contexto e a lista de fatores
                metadata.append({
                    'arquivo': arquivo_path.name,
                    'contexto': texto_contexto,
                    'fatores': fatores
                })

        except Exception as e:
            print(f"  ❌ Erro ao ler ou analisar {arquivo_path.name}: {e}")

    if not documentos:
        print("🛑 Nenhum documento válido para indexação. Verifique a pasta.")
        return None, []

    # Inicializa e treina o TF-IDF Vectorizer
    # O TF-IDF serve como um substituto simples para embeddings, capturando importância das palavras
    # Use stop_words='portuguese' para remover preposições comuns, pois aqui queremos o "conteúdo"
    # Lembre-se: idealmente, este passo usaria Sentence-Transformers, mas TF-IDF é Zero-Dependency

    vectorizer = TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=5, # Mínimo de 5 documentos para uma palavra ser relevante
        max_features=10000,
        stop_words=None # MANTEM AS STOPWORDS para a busca semântica do LLM.
    )

    # Cria o índice (matriz esparsa de vetores)
    tfidf_matrix = vectorizer.fit_transform(documentos)

    print(f"✅ Indexação TF-IDF concluída. Total de vetores: {tfidf_matrix.shape[0]}")

    return vectorizer, tfidf_matrix, metadata

def salvar_indice(vectorizer, tfidf_matrix, metadata: List[Dict[str, Any]], indice_path: Path):
    """
    Salva os componentes do índice em um arquivo JSON para facilitar o uso no Few-Shot.
    Nota: Salvar o vetorizador e a matriz TF-IDF completa é complexo em JSON,
    então vamos salvar apenas a metadata e a função de seleção será implementada em tempo real
    no script Few-Shot.
    """
    # Simplificação: Apenas salvar o metadata.
    # O Few-Shot script terá que refazer o TFIDF, ou o usuário deve salvar os objetos
    # Python serializados (pickle), o que não é recomendado em produção.

    # Para fins de demonstração, vamos apenas salvar a metadata para o próximo script carregar.
    # Em um ambiente real, o Vectorizer e a Matriz seriam serializados (com pickle/joblib)
    # ou persistidos em um banco de dados vetorial.

    # Aqui, apenas salvamos os documentos de contexto e seus fatores (os 'rótulos')
    with open(indice_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)

    print(f"💾 Metadata de treino salva em: {indice_path}")


# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================

def main_indexador():
    if not TREINO_DIR.exists():
        print(f"Diretório de treino não encontrado: {TREINO_DIR}")
        return

    # 1. Criar o índice (matriz TFIDF e metadados)
    vectorizer, tfidf_matrix, metadata = criar_indice_tfidf(TREINO_DIR)

    if metadata:
        # 2. Salvar os resultados (apenas metadata para evitar dependências complexas)
        # O próximo script (fewshot_generator.py) irá refazer a matriz TFIDF,
        # o que é aceitável para um pequeno conjunto de treino.
        salvar_indice(vectorizer, tfidf_matrix, metadata, INDICE_PATH)

if __name__ == "__main__":
    main_indexador()